# Raspberry Pi 4
> [Main Table of Contents](../README.md)

## In This Notebook
> References: [Raspberry Pi Docs](https://www.raspberrypi.com/documentation/computers/remote-access.html#setting-up-an-ssh-server)  
- Current RPi4 Function
- Use `apt` as package manager
    - Useful commands
- Enable Raspberry Pi OS SSH
- SSH headless entry into Raspberry Pi OS
- SSH entry into Raspberry Pi OS WITHOUT password
- Shutdown Pi remotely
- Setup static ip address for Rpi
- `scp` command to secure copy files from local -> remote and remote -> local using ssh tunnel

## Current RPi4 Function

> Latest Use Log: 2025.03.03

I am currently using RPi4 to house my postgres database for trading data. Especially important are the float and short float numbers.  
- Gain access to RPi using SSH (use the password version, not key version)
- There is a crontab script set up to make backups of postgres into <absolute_path>/pg_backups every weekday then send those backups to spicy Google drive. Then on every 14th day of the month at 5am, it will delete all files in <absolute_path>/pg_backups
- Aside: I have a google app script (script.google.com) in my drive that deletes backups older than 5 day every month

### Troubleshoot
- Changing Internet provider creates a new ipaddress
    - RPi has a config file to set a static ipaddress (Will need to config this). See "Setup static ip address for Rpi" below
- Check if postgres is online (see notes under postgresql)
- Check if crontab is active (also see notes under wsl2)
    - Check if Cron job is scheduled:  crontab -l
    - Check the Cron Service status: `systemctl status cron`
- Check if `rclone` token is expired with `rclone config show <nameOfDriveRegisteredWithRclon>`
    - If token is missing or invalid, delete and recreate the remote (NEED MONITOR (browser) FOR THIS; THERE IS A WAY TO DO IT HEADLESS BUT IS MORE COMPLEX)
    `rclone config delete <nameOfDriveRegisteredWithRclon>`
    `rclone config create <nameOfDriveRegisteredWithRclon>`
- Test connectivity to the remote with
    `rclone lsd <nameOfDriveRegisteredWithRclon>`



## Use `apt` as package manager

Command | Description
--- | ---
`sudo apt update`| Update all installed packages 
`sudo apt install <pck_name>` | Install packages
`dpkg --get-selections` | Get list of installed packages
`apt list --installed` | Get *detailed* list of installed packages


## Enable Raspberry Pi OS SSH
- Rasp Pi Menu dropdown -> Preferences -> Raspberry Pi Configuration -> Interfaces -> Toggle SSH

## SSH headless entry into Raspberry Pi OS using password (passwordless using keys is not setup)
- Get Rasp Pi IP address
    ```python
    # From Rasp Pi terminal
    hostname -I

    # From another computer on same network
    ping raspberrypi.local    # This is return rasp pi's ip address

    ```
- Terminal on another device
    - Username & password is that for Raspberry Pi and configured when first set up the Raspberry Pi OS
        ```
        ssh <username>@<rasp pi ip>
        ```


# SSH entry into Raspberry Pi OS WITHOUT password
- See `SSH Keys - Password-less connection betwen local machine and remote computer` section in Unix->notes.md

# Shutdown Pi Remotely
- `sudo shutdown -h now`

# Setup static ip address for Rpi
- Make sure scripts are able to access Rpi after power outages and reboots without reconfiguration of Rpi ip address in those scripts (this isn't possible with DHCP)

1. Access terminal in Rpi
    - Directly or SSH into it
2. Check if the desired ip address is already taken or not
    - `ping ip_address`
3. Get the router's ip address
    - Windows
        - `Win + R` -> `cmd` -> `ipconfig`
    - Linux
        - NOTE: Do NOT use this in WSL. Instead follow Windows instruction
        - `ip route | grep default`
4. Open Network Configuration File
    - ```sudo nano /etc/dhcpcd.conf```  
5. Add Static IP Configuration
    - Scroll down to static portion
        ```python
        # Use "eth0" if Rpi uses wire connection to router "wlan0" for Wi-Fi
        interface eth0  

        # Desired static IP address
        # The subnet mask is required. /24 subnet is common for most local networks
        # Make sure the IP address you choose is within the range of your network and not already assigned to another device
        # Note: The desired address cannot be the same one as the one its currently using
        static ip_address=192.168.1.100/24    

        # IP address of your router
        static routers=192.168.1.1  

        # DNS server (usually your router's IP)
        static domain_name_servers=192.168.1.1  
        ```
6. Restart Networking service
    - `sudo service dhcpcd restart`
7. Reboot Rpi
    - `sudo reboot`

## `scp` command to secure copy files from local -> remote and remote -> local using ssh tunnel

- secure copy file in local -> remote (PGBACKUPS dir)
    ```
    scp <filename>.txt <raspi_user>@<host>:/home/vimri/PGBACKUPS
    e.g. scp test.txt vimri@10.0.0.152:/home/vimri/PGBACKUPS
    ```
- remote -> local (current dir)
    ```
    scp <raspi_user>@<host>:/home/vimri/PGBACKUPS/<filename>.txt .
    e.g. scp vimri@10.0.0.152:/home/vimri/PGBACKUPS/text.txt .
    ```